# Training the tensorflow model on Google Cloud AI Platform

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/Breast-Cancer-Detection

In [ ]:
# without this, cv2 throws an error 
!sudo apt-get install ffmpeg libsm6 libxext6  -y

In [3]:
import os

In [4]:
PROJECT = 'vernal-buffer-285906'
REGION = 'us-west1'
BUCKET = 'vernal-buffer-285906'

In [5]:
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.3'

In [7]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [8]:
# os.environ['TRAIN_DIR'] = "/home/jupyter/Breast-Cancer-Detection/data"
# os.environ['OUTPUT_DIR'] = "/home/jupyter/Breast-Cancer-Detection/model_results"

os.environ['TRAIN_DIR'] = 'test_cnn'
os.environ['OUTPUT_DIR'] = 'gs://vernal-buffer-285906/model_results'

In [ ]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/tf_trainer
#Train using python
python -m trainer.task \
 --input-dir=$TRAIN_DIR \
 --output-dir=$OUTPUT_DIR \
 --bucket=$BUCKET

In [14]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/tf_trainer
    
OUTDIR=$OUTPUT_DIR
JOBNAME=breastcancer_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gcloud ai-platform jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/tf_trainer/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=BASIC \
  --runtime-version=2.3 \
  --python-version=3.7 \
  -- \
  --bucket=$BUCKET \
  --output-dir=$OUTDIR \
  --input-dir=$TRAIN_DIR

gs://vernal-buffer-285906/model_results us-west1 breastcancer_210303_123343
jobId: breastcancer_210303_123343
state: QUEUED


Job [breastcancer_210303_123343] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe breastcancer_210303_123343

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs breastcancer_210303_123343
